In [1]:
import keras 
from keras.datasets import mnist 
from keras.models import Sequential 
from keras.layers import Convolution2D, Dense, Flatten, MaxPooling2D
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils import to_categorical
from keras.optimizers import Adam
import os
import cv2

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)
x_test= x_test.reshape(10000, 28, 28, 1)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
for i in range(10000):
    a = "image/x_test/" + str(i) + ".bmp"
    b = "image/y_test/" + str(i)
#     np.savetxt(a, x_test[i,:,:,0])
    np.savetxt(b, y_test[i])
    cv2.imwrite(a, x_test[i])

In [3]:
model = Sequential()
model.add(Convolution2D(16, (4, 4), padding="SAME", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='SAME'))

model.add(Flatten())
model.add(Dense(units=20, activation="relu"))
model.add(Dense(units=10, activation="softmax"))
adam = Adam(lr=0.0001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

W0722 03:37:34.344120 140436407048000 deprecation_wrapper.py:119] From /home/yitao/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 03:37:34.360091 140436407048000 deprecation_wrapper.py:119] From /home/yitao/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 03:37:34.362302 140436407048000 deprecation_wrapper.py:119] From /home/yitao/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 03:37:34.380993 140436407048000 deprecation_wrapper.py:119] From /home/yitao/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0722 03:37:34.477430 1

In [5]:
model.fit(x_train, y_train, batch_size=100, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 10s 173us/step - loss: 0.9891 - acc: 0.6987
Epoch 2/5
60000/60000 [==============================] - 10s 174us/step - loss: 0.6048 - acc: 0.8226
Epoch 3/5
60000/60000 [==============================] - 10s 173us/step - loss: 0.4835 - acc: 0.8736
Epoch 4/5
60000/60000 [==============================] - 10s 175us/step - loss: 0.4136 - acc: 0.8936
Epoch 5/5
60000/60000 [==============================] - 11s 181us/step - loss: 0.3630 - acc: 0.9065


In [6]:
print(model.evaluate(x_test, y_test, batch_size=100))

10000/10000 [==============================] - 1s 79us/step
[0.3562602893076837, 0.9120000022649765]


In [7]:
model.save('model/weights.h5') #保存权重

In [8]:
model.weights

[<tf.Variable 'conv2d_1/kernel:0' shape=(4, 4, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(3136, 20) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32_ref>]

In [9]:
# for layer in model.layers:
#     weights = layer.get_weights()
#     if weights == []:
#         continue
        
#     name = layer.get_config()['name']
#     print(name)
#     directory_name = "layerData/" + name 
#     if not os.path.isdir(directory_name):
#         os.makedirs(directory_name)
        
#     if name.find('conv2d') != -1:
#         layerNum = np.shape(layer.get_weights()[0])[3]
#         imageNum = np.shape(layer.get_weights()[0])[2]
        
#         np.savetxt(directory_name + "/bias", weights[1])
#         for j in range(imageNum):
#             for i in range(layerNum):
#                 sub_dir = directory_name + "/imageNum" + str(j)
#                 if not os.path.isdir(sub_dir):
#                     os.makedirs(sub_dir)
#                 np.savetxt(sub_dir + "/" + str(i), weights[0][:,:,j,i])
#     elif name.find('dense') != -1:
#         np.savetxt(directory_name + "/weight", weights[0])
#         np.savetxt(directory_name + "/bias", weights[1])

In [10]:
f = open("layers.txt", "w")
for layer in model.layers:
    temp = layer.get_config()
    name = temp['name']
    f.write(name)
    weights = layer.get_weights()
    
    if name.find('flatten') != -1:
        f.write(" " + str(layer.get_input_shape_at(0)[3]) + "," 
                + str(layer.get_input_shape_at(0)[1]) + "," 
                + str(layer.get_input_shape_at(0)[2])) #LayDim
        f.write("\n")
        continue
        
    if name.find('max_pooling2d') != -1:
        f.write(" " + str(layer.get_input_shape_at(0)[3]) + "," 
                + str(layer.get_input_shape_at(0)[1]) + "," 
                + str(layer.get_input_shape_at(0)[2])) #LayDim
        kernel_size = temp['pool_size']
        f.write(" " + str(kernel_size[0]) + "," + str(kernel_size[1])) #maskDim
        f.write(" " + str(temp['strides'][0])) #stride
        f.write(" " + temp['padding'].upper()) #ZeroPadding
        f.write("\n")
        continue
        
    directory_name = "layerData/" + name 
    if not os.path.isdir(directory_name):
        os.makedirs(directory_name)
        
    if name.find('conv2d') != -1:
        f.write(" " + str(layer.get_input_shape_at(0)[3]) + "," 
                + str(layer.get_input_shape_at(0)[1]) + "," 
                + str(layer.get_input_shape_at(0)[2])) #LayDim
        kernel_size = temp['kernel_size']
        f.write(" " + str(temp['filters']) + "," + str(kernel_size[0]) + "," + str(kernel_size[1])) #maskDim
        f.write(" " + str(temp['strides'][0])) #stride
        f.write(" " + temp['padding'].upper()) #ZeroPadding
        f.write("\n")
        
        layerNum = np.shape(layer.get_weights()[0])[3]
        imageNum = np.shape(layer.get_weights()[0])[2]
        
        np.savetxt(directory_name + "/bias", weights[1])
        for j in range(imageNum):
            for i in range(layerNum):
                sub_dir = directory_name + "/imageNum" + str(j)
                if not os.path.isdir(sub_dir):
                    os.makedirs(sub_dir)
                np.savetxt(sub_dir + "/" + str(i), weights[0][:,:,j,i])
    elif name.find('dense') != -1:
        f.write(" " + str(layer.get_input_shape_at(0)[1]))
        f.write(" " + str(layer.get_output_shape_at(0)[1]))
        f.write(" " + str(layer.get_config()['activation']))
        f.write("\n")
        np.savetxt(directory_name + "/weight", weights[0])
        np.savetxt(directory_name + "/bias", weights[1])
f.close()